In [3]:
import tensorflow as tf
from beyondml import tflow

from sklearn.metrics import classification_report, confusion_matrix

def print_results(truth, preds):
    print(confusion_matrix(truth, preds))
    print(classification_report(truth, preds))
    print('\n\n')

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = tf.keras.applications.resnet50.preprocess_input(x_train)
x_test = tf.keras.applications.resnet50.preprocess_input(x_test)

In [5]:
input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = tf.keras.layers.UpSampling2D((2, 2))(input_layer)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.applications.ResNet50(include_top = False)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MultiMaskedDense(128, activation = 'relu')([x, x, x,])
x1 = tflow.layers.SelectorLayer(0)(x)
x2 = tflow.layers.SelectorLayer(1)(x)
x3 = tflow.layers.SelectorLayer(2)(x)
x1 = tf.keras.layers.Dropout(0.5)(x1)
x2 = tf.keras.layers.Dropout(0.5)(x2)
x3 = tf.keras.layers.Dropout(0.5)(x3)
x1 = tf.keras.layers.BatchNormalization()(x1)
x2 = tf.keras.layers.BatchNormalization()(x2)
x3 = tf.keras.layers.BatchNormalization()(x3)
x = tflow.layers.MultiMaskedDense(64, activation = 'relu')([x1, x2, x3])
x1 = tflow.layers.SelectorLayer(0)(x)
x2 = tflow.layers.SelectorLayer(1)(x)
x3 = tflow.layers.SelectorLayer(2)(x)
x1 = tf.keras.layers.Dropout(0.5)(x1)
x2 = tf.keras.layers.Dropout(0.5)(x2)
x3 = tf.keras.layers.Dropout(0.5)(x3)
x1 = tf.keras.layers.BatchNormalization()(x1)
x2 = tf.keras.layers.BatchNormalization()(x2)
x3 = tf.keras.layers.BatchNormalization()(x3)
output_layer = tflow.layers.MultiMaskedDense(10, activation = 'softmax')([x1, x2, x3])

model = tf.keras.models.Model(input_layer, output_layer)
model = tflow.utils.add_layer_masks(model)
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = tf.keras.optimizers.Adamax())

2022-11-09 14:11:44.188621: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 14:11:44.867106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11553 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


In [6]:
model.fit(
    x_train,
    [y_train, y_train, y_train],
    batch_size = 64,
    epochs = 2,
    validation_split = 0.2
)

Epoch 1/2
625/625 [==============================] - 1042s 2s/step - loss: 3.7810 - multi_masked_dense_5_loss: 1.2644 - multi_masked_dense_5_1_loss: 1.2641 - multi_masked_dense_5_2_loss: 1.2525 - multi_masked_dense_5_accuracy: 0.6160 - multi_masked_dense_5_1_accuracy: 0.6183 - multi_masked_dense_5_2_accuracy: 0.6237 - val_loss: 2.1011 - val_multi_masked_dense_5_loss: 0.7027 - val_multi_masked_dense_5_1_loss: 0.6891 - val_multi_masked_dense_5_2_loss: 0.7092 - val_multi_masked_dense_5_accuracy: 0.7956 - val_multi_masked_dense_5_1_accuracy: 0.7959 - val_multi_masked_dense_5_2_accuracy: 0.7916
Epoch 2/2
625/625 [==============================] - 1026s 2s/step - loss: 1.8260 - multi_masked_dense_5_loss: 0.6083 - multi_masked_dense_5_1_loss: 0.6067 - multi_masked_dense_5_2_loss: 0.6111 - multi_masked_dense_5_accuracy: 0.8306 - multi_masked_dense_5_1_accuracy: 0.8334 - multi_masked_dense_5_2_accuracy: 0.8333 - val_loss: 1.8244 - val_multi_masked_dense_5_loss: 0.6070 

2022-11-09 14:12:04.031762: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2022-11-09 14:12:04.358186: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-09 14:12:06.959452: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [7]:
model = tflow.utils.mask_model(model, 70, method = 'magnitude')
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = tf.keras.optimizers.Adamax())

callbacks = [
    tf.keras.callbacks.EarlyStopping(min_delta = 0.001, patience = 5, restore_best_weights = True),
    tf.keras.callbacks.ReduceLROnPlateau(min_lr = 1e-8, patience = 3)
]
model.fit(
    x_train,
    [y_train, y_train, y_train],
    batch_size = 64,
    epochs = 100,
    validation_split = 0.2,
    callbacks = callbacks
)

model = tflow.utils.remove_layer_masks(model)

preds = model.predict(x_test)

print_results(y_test, preds[0].argmax(axis = 1))
print_results(y_test, preds[1].argmax(axis = 1))
print_results(y_test, preds[2].argmax(axis = 1))
print_results(y_test, sum(preds).argmax(axis = 1))

model.save('sparse_resnet.h5')

Epoch 1/100
625/625 [==============================] - 1037s 2s/step - loss: 2.0995 - multi_masked_dense_5_loss: 0.5153 - multi_masked_dense_5_1_loss: 0.7222 - multi_masked_dense_5_2_loss: 0.8620 - multi_masked_dense_5_accuracy: 0.8802 - multi_masked_dense_5_1_accuracy: 0.8274 - multi_masked_dense_5_2_accuracy: 0.7929 - val_loss: 1.0113 - val_multi_masked_dense_5_loss: 0.2916 - val_multi_masked_dense_5_1_loss: 0.3346 - val_multi_masked_dense_5_2_loss: 0.3851 - val_multi_masked_dense_5_accuracy: 0.9150 - val_multi_masked_dense_5_1_accuracy: 0.9135 - val_multi_masked_dense_5_2_accuracy: 0.9146
Epoch 2/100
625/625 [==============================] - 1019s 2s/step - loss: 1.2591 - multi_masked_dense_5_loss: 0.3029 - multi_masked_dense_5_1_loss: 0.4264 - multi_masked_dense_5_2_loss: 0.5298 - multi_masked_dense_5_accuracy: 0.9309 - multi_masked_dense_5_1_accuracy: 0.8980 - multi_masked_dense_5_2_accuracy: 0.8687 - val_loss: 0.7536 - val_multi_masked_dense_5_loss: 0.2